In [1]:
!pip install transformers


In [2]:
!pip install tensorboardx

In [3]:
!pip install simpletransformers


In [4]:
!pip install scikit-learn

In [6]:
import pandas as pd


#train_df = pd.read_csv('data/train.csv', header=None)
df = pd.read_csv('./data/newsdata.csv', sep='\t', names=['ID','TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])

In [7]:
df = df[['TITLE','CATEGORY']]
#train_df['text'] = train_df.iloc[:, 1] + " " + train_df.iloc[:, 2]

In [8]:
encode_dict = {}

def encode_cat(x):
    if x not in encode_dict.keys():
        encode_dict[x]=len(encode_dict)
    return encode_dict[x]

df['ENCODE_CAT'] = df['CATEGORY'].apply(lambda x: encode_cat(x))

#train_df = train_df.drop(train_df.columns[[1, 2]], axis=1)

In [ ]:
train_df.columns = ['label', 'text']

In [ ]:
train_df = train_df[['text', 'label']]

In [ ]:
train_df['text'] = train_df['text'].apply(lambda x: x.replace('\\', ' '))

In [ ]:
train_df['label'] = train_df['label'].apply(lambda x:x-1)

In [ ]:
train_df

In [ ]:
eval_df = pd.read_csv('data/test.csv', header=None)
eval_df['text'] = eval_df.iloc[:, 1] + " " + eval_df.iloc[:, 2]
eval_df = eval_df.drop(eval_df.columns[[1, 2]], axis=1)
eval_df.columns = ['label', 'text']
eval_df = eval_df[['text', 'label']]
eval_df['text'] = eval_df['text'].apply(lambda x: x.replace('\\', ' '))
eval_df['label'] = eval_df['label'].apply(lambda x:x-1)

In [ ]:
eval_df

In [ ]:
from simpletransformers.classification import ClassificationModel


In [ ]:
model = ClassificationModel('distilbert', 'distilbert-base-uncased', use_cuda=False, num_labels=4)


In [ ]:
model.train_model(train_df)

In [ ]:
#result, model_outputs, wrong_predictions = model.eval_model(eval_df)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df, f1=f1_multiclass, acc=accuracy_score)

In [ ]:
result

In [ ]:
model_outputs

In [ ]:
wrong_predictions

In [ ]:
predictions, raw_outputs = model.predict(['Explosions Echo Throughout Najaf","NAJAF, Iraq - U.S. tanks and troops rolled back into the center of Najaf and battled with Shiite militants Sunday, reigniting violence in the holy city just as delegates in Baghdad opened a conference meant to be a landmark in the countrys movement toward democracy...'])

In [ ]:
predictions